In [65]:
import pandas as pd
import numpy as np
import json

In [47]:

def generate_info(dataid, name, type, subtype, subtype2, rate, size, uint,
                  notes):
    data = {
        name: {
            "data_id": dataid,
            "data_name": name,
            "data_type": type,
            "data_subtype1": subtype,
            "data_subtype2": subtype2,
            "data_rate": rate,
            "data_size": size,
            "data_unit": uint,
            "data_notes": notes
        }
    }
    return data


In [34]:
data = pd.read_csv("Failures Modes in MCVT v6 - Failure Modes of MCVT.csv")

In [36]:
data['shape'] = data.groupby('Component ID').transform('count')['System']

In [37]:
data['Component Fault'] = data['Component Fault'].astype(str)

In [39]:
data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']

/var/folders/jg/vffvcnyd6_bfssm4fcwph6b00000gn/T/ipykernel_22369/3851792186.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']
/var/folders/jg/vffvcnyd6_bfssm4fcwph6b00000gn/T/ipykernel_22369/3851792186.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  data['fault'] = data.groupby('Component ID').transform(lambda x: ','.join(x))['Component Fault']
/var/folders/jg/vffvcnyd6_bfssm4fcwph6b00000gn/T/ipykernel_22369/3851792186.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeEr

In [41]:
data['System'].unique()

array(['ECLSS', 'Structure', 'SPL', 'Disturbance', 'Interior Environment',
       'Power'], dtype=object)

In [43]:
trans = {
    'ECLSS': 4,
    'Structure': 2,
    'SPL' : 8,
    'Disturbance' : 9,
    'Interior Environment' : 6,
    'Power': 3,
}

In [45]:
row['Activity Type']

TypeError: tuple indices must be integers or slices, not str

In [62]:
result = {}

for _, row in data.iterrows():
    if row['Activity Type'] == 3:
        dataid = row['Component ID']
        name = row['System/Sub-system/Component']
        dtype = 2
        subtype = trans[row['System']]
        subtype2 = 255
        rate = 1
        size = row['shape']
        unit = 'n/a'
        notes = row['fault']
        result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))
    if row['Activity Type'] == 1:
        dataid = row['Unnamed: 3'] + 128
        name = row['System/Sub-system/Component'] + '/' + row['Component Fault']
        dtype = 1
        subtype = trans[row['System']]
        subtype2 = 255
        rate = 1
        size = int(row['Signal Size'].split('x')[0]) if isinstance(row['Signal Size'], str) else row['Signal Size']
        unit = 'n/a'
        notes = row['fault']
        result.update(generate_info(dataid, name, dtype, subtype, subtype2, rate, size, unit, notes))

In [66]:
json_string = json.dumps(result)
with open('db_info_v6.json', 'w') as outfile:
    outfile.write(json_string)